In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3' # nvidia-smi로 비어있는 gpu 확인하고 여기서 선택할것!


#from huggingface_hub import notebook_login

#notebook_login()

In [2]:
import json
import pandas as pd
from copy import deepcopy
import random
from tqdm.notebook import tqdm

from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    AdamW
)
import pickle

from asset.tokenization_kobert import KoBertTokenizer
from asset import tokenization_kobert
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [3]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [4]:
def split_input_dict(input_dict, ratio = 0.01, seed = 42):
    split_point = int(len(input_dict['utterance']) * ratio)
    random.seed(seed)
    random.shuffle(input_dict['utterance'])
    valid_dict = deepcopy(input_dict)
    train_dict = input_dict

    valid_dict['utterance'] = input_dict['utterance'][:split_point]
    train_dict['utterance'] = input_dict['utterance'][split_point:]
    return train_dict, valid_dict

In [5]:
def read_input(path):
    data = pd.read_csv(path)
    input_dict = {"utterance":[]}
    for _, row in tqdm(data.iterrows()):
        for q_col in ['질문_1', '질문_2']:
            for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
                input_dict["utterance"].append({"question":row[q_col],"answer":row[a_col]})
            
            
    train_dict,valid_dict = split_input_dict(input_dict,ratio=0)
    train_question = []
    train_answer = []
    for pairs in tqdm(train_dict['utterance']):
        train_question.append(pairs['question'])
        train_answer.append(pairs['answer'])

    valid_question = []
    valid_answer = []
    for pairs in tqdm(valid_dict['utterance']):
        valid_question.append(pairs['question'])
        valid_answer.append(pairs['answer'])

    
    return train_question ,train_answer , valid_question , valid_answer

In [6]:
train_q ,train_a , valid_q , valid_a = read_input("./bigdata/train.csv")

0it [00:00, ?it/s]

  0%|          | 0/6440 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [7]:
####데이터 3개씩 끊어보는 기능
n=2
for ind,i in enumerate(zip(train_q,train_a)):
    if ind>3:
        break
    print(i[0]+" ## "+i[1])

평지붕이 뭐야? ## 평지붕은 경사가 없는 지붕을 의미합니다. 주로 건물의 상부 층이나 테라스, 차고, 창고 등에 사용되는데, 지붕이 평평하기 때문에 시공이 비교적 간단하고 활용도가 높습니다. 평지붕은 옥외 휴식 공간이나 태양광 패널, 에어컨 등을 설치하기에도 용이하며, 건물의 외관을 모던하고 스마트하게 연출할 수 있어 인기가 있습니다. 다만, 비가 내리거나 눈이 쌓일 경우 배수가 어렵기 때문에 지붕 내부에 방수 시스템을 설치하는 것이 중요합니다.
폴리싱 타일을 사용하는 장점은 무엇인가요? ## 폴리싱 타일은 내구성이 좋고 변형 및 오염, 얼룩에 강해 반영구적으로 사용이 가능하다는 장점이 있습니다.
도어 프레임이 변형되어 문이 제대로 닫히지 않을 때 어떻게 해결하나요? ## 도어 프레임이 변형되어 문이 제대로 닫히지 않을 때, 먼저 프레임을 조정하여 수평으로 맞춰보실 수 있습니다. 이를 위해 프레임 주변의 공간을 측정하고 변형된 부분을 확인한 후 조정 작업을 진행하시면 됩니다. 다만, 이는 전문가가 아닌 경우에는 어려울 수 있으므로 전문가의 도움을 받는 것이 좋습니다. 또한, 만일 프레임을 조정 후에도 문이 제대로 닫히지 않는다면, 문을 다시 설치하거나 교체하는 것을 고려해 보실 수 있습니다. 프레임이 심각하게 변형되었다면 전문가와 상의하여 신중하게 결정하는 것이 중요합니다.
집 안팎에 실내-외 연계를 만드는 데 사용할 수 있는 방법은 어떤 것이 있을까요? ## 실내-외 연계를 만들기 위해서는 패턴 매칭과 넓은 유리 도어, 그리고 야외 가구를 활용하는 것이 좋습니다. 또한, 동일한 색상 팔레트를 사용하여 내부와 외부를 연결시키고, 실내 공간과 야외 공간을 시각적으로 연결하는 것이 중요합니다. 이를 통해 내부와 외부의 경계를 흐림으로써 공간 간의 일관성을 유지할 수 있습니다. 특히 실내 및 외부 바닥재, 가구 및 장식품의 설계 및 장식을 통해 일관된 분위기를 조성할 수 있습니다.


In [8]:
class PairedDataset:
    def __init__(self, src_tokenizer, tgt_tokenizer,file_question , file_answer):
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = tgt_tokenizer
        self.data_q = file_question
        self.data_a = file_answer
        self.data = [[self.data_q[i],self.data_a[i]] for i in range(len(self.data_q)) ]
    def __getitem__(self, index):
        src, trg = self.data_q[index] , self.data_a[index]
        embeddings = self.src_tokenizer(src, return_attention_mask=False, return_token_type_ids=False)
        embeddings['labels'] = self.trg_tokenizer(trg, return_attention_mask=False)['input_ids']

        return embeddings

    def __len__(self):
        return len(self.data)

In [9]:
dataset = PairedDataset(src_tokenizer, trg_tokenizer, train_q , train_a)
eval_dataset = PairedDataset(src_tokenizer, trg_tokenizer, valid_q , valid_a)

In [10]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    'monologg/kobert',
    'skt/kogpt2-base-v2',
    pad_token_id=trg_tokenizer.bos_token_id
)





model.config.decoder_start_token_id = trg_tokenizer.bos_token_id
model

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['transformer.h.1.crossattention.q_attn.bias', 'transformer.h.6.crossattention.c_attn.weight', 'transformer.h.3.crossattention.c_attn.bias', 'transformer.h.11.crossattention.c_proj.bias', 'transformer.h.0.crossattention.c_attn.weight', 'transformer.h.3.ln_cross_attn.weight', 'transformer.h.0.crossattention.c_proj.weight', 'transformer.h.5.crossattention.c_proj.bias', 'transformer.h.8.ln_cross_attn.bias', 'transformer.h.4.ln_cross_attn.bias', 'transformer.h.1.crossattention.q_attn.weight', 'transformer.h.5.crossattention.q_attn.bias', 'transformer.h.3.crossattention.c_proj.bias', 'transformer.h.3.ln_cross_attn.bias', 'transformer.h.11.crossattention.c_proj.weight', 'transformer.h.4.crossattention.q_attn.weight', 'transformer.h.6.crossattention.c_attn.bias', 'transformer.h.7.crossattention.q_attn.weight', 'transformer.h.0.crossattention.q_attn.bias', 'transforme

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [14]:
collator = DataCollatorForSeq2Seq(src_tokenizer, model)

arguments = Seq2SeqTrainingArguments(
    output_dir='dacon-v0',
    do_train=True,
    #do_eval=True,
    #evaluation_strategy="steps",
    #eval_steps=6000,
    save_strategy="steps",
    save_steps=6000,
    num_train_epochs=120,
    per_device_train_batch_size=16,
    #per_device_eval_batch_size=16,
    warmup_ratio=0.1,
    gradient_accumulation_steps=1,
    save_total_limit=5,
    dataloader_num_workers=1,
    fp16=True,
    #load_best_model_at_end=True,
    #push_to_hub=True,
    learning_rate = 5e-5
)

trainer = Seq2SeqTrainer(
    model,
    arguments,
    data_collator=collator,
    train_dataset=dataset,
    #eval_dataset=eval_dataset
)

In [15]:
trainer.train()

#model.save_pretrained("checkpoints/dacon-v0/best_model")

Step,Training Loss
500,3.487800
1000,2.662500
1500,2.029500
2000,1.567200
2500,1.193200
3000,0.930900
3500,0.728900
4000,0.579500
4500,0.459600
5000,0.387600


/home/leadawon5/gitfiles/venvs/myvenv/lib/python3.7/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/home/leadawon5/gitfiles/venvs/myvenv/lib/python3.7/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decode

TrainOutput(global_step=48360, training_loss=0.19781624643737566, metrics={'train_runtime': 9493.1018, 'train_samples_per_second': 81.406, 'train_steps_per_second': 5.094, 'total_flos': 2.517036228937728e+16, 'train_loss': 0.19781624643737566, 'epoch': 120.0})

In [13]:
#trainer.push_to_hub()